# 8장. 프로덕션 기능

In [ ]:
!pip install langchain langchain-community langchain-openai langgraph

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

---
## 코드 8-1~8-2 Structured Output: 구조화된 출력

# Structured Output: 구조화된 출력

이 노트북에서는 **LLM의 출력을 특정 구조로 강제**하는 방법을 배웁니다.

## Structured Output이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Structured Output의 개념                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 LLM 출력:                                                   │
│   "고양이 농담이요? 음... 고양이가 컴퓨터를 좋아하는 이유는..."    │
│   → 자유 형식 텍스트 (파싱하기 어려움)                            │
│                                                                    │
│   Structured Output:                                               │
│   {                                                                │
│     "setup": "고양이가 컴퓨터를 좋아하는 이유는?",               │
│     "punchline": "마우스가 있으니까!"                            │
│   }                                                                │
│   → 정해진 구조로 출력 (파싱 용이, 타입 안전)                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 왜 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Structured Output 사용 사례                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   • API 응답 생성: 특정 JSON 형식 필요                            │
│   • 데이터 추출: 이름, 날짜, 금액 등 구조화된 정보 추출           │
│   • 분류 작업: 정해진 카테고리 중 하나 선택                       │
│   • 에이전트 결정: 다음 행동 선택 (Supervisor 패턴)               │
│   • 폼 데이터 생성: 사용자 입력 검증                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. Pydantic 스키마 정의

출력 구조를 Pydantic 모델로 정의합니다.

In [ ]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    """
    농담 구조
    
    setup: 농담의 설정 ("왜 고양이가...")
    punchline: 농담의 포인트 ("마우스가 있으니까!")
    """
    setup: str = Field(description='농담의 설정')
    punchline: str = Field(description='농담의 포인트')

print("✅ Joke 스키마 정의 완료")
print("   - setup: 농담의 설정")
print("   - punchline: 농담의 포인트")

# 3. LLM에 Structured Output 적용

In [ ]:
from langchain_ollama import ChatOllama

# 기본 모델
base_model = ChatOllama(model='llama3.2', temperature=0)

# Structured Output 적용
model = base_model.with_structured_output(Joke)

print("✅ Structured Output LLM 설정 완료")
print("   항상 Joke 형태로 응답함")

# 4. 실행 및 결과 확인

In [ ]:
# 농담 생성
result = model.invoke('고양이에 대한 농담을 만들어 주세요.')

print("=== Structured Output 결과 ===")
print(f"타입: {type(result)}")
print(f"\n결과: {result}")
print(f"\n설정: {result.setup}")
print(f"포인트: {result.punchline}")

# 5. 다양한 스키마 예시

In [ ]:
from typing import Literal, Optional
from pydantic import BaseModel, Field

# 예시 1: 감정 분석
class SentimentAnalysis(BaseModel):
    sentiment: Literal['positive', 'negative', 'neutral'] = Field(
        description='텍스트의 감정'
    )
    confidence: float = Field(
        description='신뢰도 (0.0 ~ 1.0)',
        ge=0.0,
        le=1.0
    )
    reason: str = Field(description='판단 이유')

# 예시 2: 개체명 인식
class Entity(BaseModel):
    name: str = Field(description='개체 이름')
    type: Literal['person', 'organization', 'location', 'date', 'other'] = Field(
        description='개체 유형'
    )

class EntityExtraction(BaseModel):
    entities: list[Entity] = Field(description='추출된 개체 목록')

# 예시 3: 작업 분류
class TaskClassification(BaseModel):
    category: Literal['question', 'request', 'complaint', 'feedback'] = Field(
        description='작업 카테고리'
    )
    priority: Literal['low', 'medium', 'high', 'urgent'] = Field(
        description='우선순위'
    )
    summary: str = Field(description='요약')

print("✅ 다양한 스키마 정의 완료")

In [ ]:
# 감정 분석 테스트
sentiment_model = base_model.with_structured_output(SentimentAnalysis)

text = "이 제품 정말 최고예요! 배송도 빠르고 품질도 좋습니다."
result = sentiment_model.invoke(f"다음 텍스트의 감정을 분석해주세요: {text}")

print("=== 감정 분석 결과 ===")
print(f"텍스트: {text}")
print(f"\n감정: {result.sentiment}")
print(f"신뢰도: {result.confidence}")
print(f"이유: {result.reason}")

In [ ]:
# 개체명 인식 테스트
entity_model = base_model.with_structured_output(EntityExtraction)

text = "삼성전자의 이재용 회장이 2024년 1월 서울에서 기자회견을 열었습니다."
result = entity_model.invoke(f"다음 텍스트에서 개체를 추출해주세요: {text}")

print("=== 개체명 인식 결과 ===")
print(f"텍스트: {text}")
print(f"\n추출된 개체:")
for entity in result.entities:
    print(f"  - {entity.name} ({entity.type})")

---

## 정리: Structured Output

### 핵심 코드

```python
from pydantic import BaseModel, Field

# 1. 스키마 정의
class MyOutput(BaseModel):
    field1: str = Field(description='설명')
    field2: int = Field(description='설명')

# 2. LLM에 적용
model = base_model.with_structured_output(MyOutput)

# 3. 실행 (결과가 MyOutput 타입)
result = model.invoke('프롬프트')
print(result.field1)  # 타입 안전
```

### Field 옵션

| 옵션 | 설명 | 예시 |
|------|------|------|
| **description** | 필드 설명 | `Field(description='이름')` |
| **default** | 기본값 | `Field(default='기본')` |
| **ge/le** | 숫자 범위 | `Field(ge=0, le=100)` |
| **min_length** | 문자열 최소 길이 | `Field(min_length=1)` |

### Literal로 선택지 제한

```python
from typing import Literal

class Decision(BaseModel):
    choice: Literal['option1', 'option2', 'option3']  # 이 중 하나만 가능
```

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
model = model.with_structured_output(Joke)

# 변경
model = ChatOllama(model='llama3.2', temperature=0)
model = model.with_structured_output(Joke)
```

## 다음 단계

**Streaming**을 사용하여 실시간으로 출력을 받는 방법을 배웁니다. (03번 노트북)

---
## 코드 8-3 Streaming Output: 노드별 스트리밍

# Streaming Output: 노드별 스트리밍

이 노트북에서는 **그래프 실행 중 노드별로 결과를 스트리밍**하는 방법을 배웁니다.

## Streaming이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Streaming의 개념                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 실행 (invoke):                                              │
│   입력 → [처리 중...] → 전체 결과 반환                             │
│   (모든 처리가 끝날 때까지 기다림)                                 │
│                                                                    │
│   스트리밍 실행 (stream):                                          │
│   입력 → 노드1 결과 → 노드2 결과 → 노드3 결과 → ...               │
│   (각 노드 완료 시마다 즉시 결과 반환)                             │
│                                                                    │
│   장점:                                                            │
│   • 실시간 진행 상황 확인                                          │
│   • 사용자 경험 향상 (응답성)                                      │
│   • 디버깅 용이                                                    │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## stream_mode 옵션

```
┌────────────────────────────────────────────────────────────────────┐
│                    stream_mode 비교                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   stream_mode='values' (기본):                                     │
│   → 각 노드 후 전체 State 반환                                    │
│                                                                    │
│   stream_mode='updates':                                           │
│   → 각 노드가 변경한 부분만 반환                                  │
│   → 더 효율적, 변경 사항 추적 용이                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)


class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state['messages'])
    return {'messages': res}


builder = StateGraph(State)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile()

print("✅ 에이전트 그래프 준비 완료")

# 3. 기본 스트리밍 (values 모드)

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== stream_mode='values' (기본) ===")
print("각 노드 후 전체 State 반환\n")

for i, chunk in enumerate(graph.stream(input_data)):
    print(f"--- Chunk {i+1} ---")
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            content = str(last_msg.content)[:100] if hasattr(last_msg, 'content') else str(last_msg)[:100]
            print(f"내용: {content}...")
    print()

# 4. updates 모드 스트리밍

In [ ]:
print("=== stream_mode='updates' ===")
print("각 노드가 변경한 부분만 반환\n")

for i, chunk in enumerate(graph.stream(input_data, stream_mode='updates')):
    print(f"--- Update {i+1} ---")
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        print(f"변경 내용: {str(node_output)[:200]}...")
    print()

# 5. 실시간 진행 상황 표시

In [ ]:
import time

print("=== 실시간 진행 상황 ===")
print(f"질문: {input_data['messages'][0].content}\n")

start_time = time.time()

for chunk in graph.stream(input_data, stream_mode='updates'):
    elapsed = time.time() - start_time
    
    for node_name, node_output in chunk.items():
        if node_name == 'model':
            msg = node_output.get('messages', [])[-1] if node_output.get('messages') else None
            if msg and hasattr(msg, 'tool_calls') and msg.tool_calls:
                print(f"[{elapsed:.1f}s] 🔧 도구 호출 결정: {[tc['name'] for tc in msg.tool_calls]}")
            elif msg and hasattr(msg, 'content') and msg.content:
                print(f"[{elapsed:.1f}s] 💬 최종 응답 생성 중...")
        elif node_name == 'tools':
            print(f"[{elapsed:.1f}s] ⚡ 도구 실행 완료")

print(f"\n총 소요 시간: {time.time() - start_time:.1f}초")

---

## 정리: Streaming Output

### 핵심 코드

```python
# 기본 스트리밍 (전체 State)
for chunk in graph.stream(input_data):
    print(chunk)

# updates 모드 (변경 사항만)
for chunk in graph.stream(input_data, stream_mode='updates'):
    print(chunk)
```

### stream_mode 비교

| 모드 | 반환 내용 | 사용 시나리오 |
|------|----------|---------------|
| **values** | 전체 State | 전체 상태 필요 시 |
| **updates** | 변경된 부분만 | 효율적인 처리, 변경 추적 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**토큰 단위 스트리밍**으로 더 세밀한 실시간 출력을 구현합니다. (04번 노트북)

---
## 코드 8-4 Token Streaming: 토큰 단위 스트리밍

# Token Streaming: 토큰 단위 스트리밍

이 노트북에서는 **LLM 출력을 토큰 단위로 실시간 스트리밍**하는 방법을 배웁니다.

## 토큰 스트리밍이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    토큰 스트리밍의 개념                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   노드 단위 스트리밍:                                              │
│   노드 완료 → 전체 응답 반환                                       │
│   "미국의 30대 대통령은 캘빈 쿨리지입니다..." (전체)              │
│                                                                    │
│   토큰 단위 스트리밍:                                              │
│   "미국" → "의" → "30" → "대" → "대통령" → "은" → ...          │
│   (실시간으로 글자가 나타남)                                       │
│                                                                    │
│   장점:                                                            │
│   • ChatGPT처럼 실시간 타이핑 효과                                 │
│   • 사용자가 기다리는 느낌 감소                                    │
│   • 긴 응답도 즉시 시작                                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)


class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state['messages'])
    return {'messages': res}


builder = StateGraph(State)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

# checkpointer 필요 (astream_events 사용 시)
graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. 토큰 스트리밍 (astream_events)

비동기 방식으로 토큰 단위 스트리밍을 구현합니다.

In [ ]:
import asyncio

async def stream_tokens():
    """
    astream_events를 사용한 토큰 스트리밍
    
    event['event'] == 'on_chat_model_stream' 일 때
    토큰 단위로 content를 받을 수 있음
    """
    input_data = {
        'messages': [
            HumanMessage(
                '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
            )
        ]
    }
    config = {'configurable': {'thread_id': '1'}}
    
    print("=== 토큰 스트리밍 ===")
    print(f"질문: {input_data['messages'][0].content}\n")
    print("응답: ", end='', flush=True)
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        # LLM 스트리밍 이벤트 캡처
        if event["event"] == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end='', flush=True)  # 토큰 단위 출력
    
    print("\n\n✅ 스트리밍 완료")

# 비동기 실행
await stream_tokens()

# 4. 이벤트 유형 확인

In [ ]:
async def show_events():
    """
    다양한 이벤트 유형 확인
    """
    input_data = {
        'messages': [
            HumanMessage('안녕하세요, 간단히 인사해주세요.')
        ]
    }
    config = {'configurable': {'thread_id': '2'}}
    
    print("=== 이벤트 유형 ===")
    
    event_types = set()
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        event_type = event["event"]
        if event_type not in event_types:
            event_types.add(event_type)
            print(f"새 이벤트 유형: {event_type}")
    
    print(f"\n발견된 이벤트 유형: {len(event_types)}개")

await show_events()

# 5. 주요 이벤트 필터링

In [ ]:
async def filtered_stream():
    """
    주요 이벤트만 필터링하여 표시
    """
    input_data = {
        'messages': [
            HumanMessage('파이썬이란 무엇인가요? 한 문장으로 설명해주세요.')
        ]
    }
    config = {'configurable': {'thread_id': '3'}}
    
    print("=== 필터링된 이벤트 스트림 ===")
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        event_type = event["event"]
        
        if event_type == "on_chain_start":
            name = event.get("name", "unknown")
            print(f"🚀 체인 시작: {name}")
            
        elif event_type == "on_chat_model_start":
            print(f"🤖 LLM 호출 시작")
            
        elif event_type == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(f"📝 토큰: '{content}'")
                
        elif event_type == "on_chat_model_end":
            print(f"✅ LLM 호출 완료")
            
        elif event_type == "on_chain_end":
            name = event.get("name", "unknown")
            if name == "LangGraph":
                print(f"🏁 그래프 완료")

await filtered_stream()

---

## 정리: Token Streaming

### 핵심 코드

```python
import asyncio

async def stream_tokens():
    async for event in graph.astream_events(input, config, version="v2"):
        if event["event"] == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end='', flush=True)

# 실행
await stream_tokens()  # 또는 asyncio.run(stream_tokens())
```

### 주요 이벤트 유형

| 이벤트 | 설명 | 용도 |
|--------|------|------|
| **on_chat_model_stream** | LLM 토큰 출력 | 실시간 응답 표시 |
| **on_chat_model_start** | LLM 호출 시작 | 로딩 표시 |
| **on_chat_model_end** | LLM 호출 완료 | 로딩 종료 |
| **on_chain_start/end** | 체인 시작/완료 | 진행 상황 추적 |
| **on_tool_start/end** | 도구 시작/완료 | 도구 실행 추적 |

### 주의 사항

- `astream_events`는 비동기 함수 (`async for` 필요)
- `checkpointer`가 필요함 (MemorySaver 등)
- `version="v2"` 권장

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**Interrupt**를 사용하여 그래프 실행을 중단하고 재개하는 방법을 배웁니다. (05-06번 노트북)

---
## 코드 8-5~8-6 Interrupt: 그래프 실행 중단 및 재개

# Interrupt: 그래프 실행 중단 및 재개

이 노트북에서는 **그래프 실행을 특정 지점에서 중단하고 재개**하는 방법을 배웁니다.

## Interrupt란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Interrupt의 개념                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 실행:                                                       │
│   START → 노드1 → 노드2 → 노드3 → END                             │
│   (중간에 멈출 수 없음)                                            │
│                                                                    │
│   interrupt_before 사용:                                           │
│   START → 노드1 → [중단] → 사용자 승인 → 노드2 → 노드3 → END     │
│                                                                    │
│   사용 사례:                                                       │
│   • 도구 실행 전 사용자 승인                                       │
│   • 결제/삭제 등 위험한 작업 전 확인                               │
│   • Human-in-the-loop 워크플로우                                   │
│   • 디버깅 및 검토                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Interrupt 동작 흐름                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ 첫 번째 실행                                                 │
│   graph.stream(input, config, interrupt_before=["model"])         │
│   → select_tools 실행 → model 전에 중단 → 상태 저장               │
│                                                                    │
│   2️⃣ 사용자 검토/승인                                             │
│   (여기서 상태 확인, 수정 가능)                                    │
│                                                                    │
│   3️⃣ 두 번째 실행 (재개)                                          │
│   graph.stream(None, config)  # None = 저장된 상태에서 재개       │
│   → model 실행 → tools 실행 → ... → 완료                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={'name': tool.name}) for tool in tools],
    embeddings,
).as_retriever()


class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


def model_node(state: State) -> State:
    selected_tools = [tool for tool in tools if tool.name in state['selected_tools']]
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    return {'messages': res}


def select_tools(state: State) -> State:
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}


builder = StateGraph(State)
builder.add_node('select_tools', select_tools)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'select_tools')
builder.add_edge('select_tools', 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

# MemorySaver 필수 (상태 저장용)
graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. interrupt_before 사용

특정 노드 실행 **전에** 중단합니다.

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
        )
    ]
}

config = {"configurable": {"thread_id": "1"}}

print("=== 첫 번째 실행 (model 전에 중단) ===")
print(f"질문: {input_data['messages'][0].content}\n")

# interrupt_before=["model"]: model 노드 실행 전에 중단
output = graph.stream(input_data, config, interrupt_before=["model"])

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        print(f"  출력: {node_output}")
    print()

print("🛑 model 노드 전에 중단됨")

# 4. 중단된 상태 확인

In [ ]:
# 현재 저장된 상태 확인
state = graph.get_state(config)

print("=== 중단된 상태 ===")
print(f"메시지: {state.values.get('messages', [])}")
print(f"선택된 도구: {state.values.get('selected_tools', [])}")
print(f"\n다음 실행할 노드: {state.next}")

# 5. 재개 (Resume)

In [ ]:
print("=== 두 번째 실행 (재개) ===")
print("사용자가 승인했다고 가정\n")

# None을 전달하면 저장된 상태에서 재개
output = graph.stream(None, config)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

print("✅ 완료")

# 6. 새 입력으로 재개

In [ ]:
# 새로운 스레드로 시작
config2 = {"configurable": {"thread_id": "2"}}

print("=== 첫 번째 실행 (중단) ===")
output = graph.stream(input_data, config2, interrupt_before=["model"])
for chunk in output:
    pass  # 출력 생략
print("🛑 중단됨\n")

# 새 입력으로 재개 (질문 변경)
new_input = {
    'messages': [
        HumanMessage(
            '영화 보이후드의 촬영 기간은 얼마나 되나요?'
        )
    ]
}

print("=== 두 번째 실행 (새 입력으로 재개) ===")
print(f"새 질문: {new_input['messages'][0].content}\n")

# 새 입력 전달 (기존 상태에 추가됨)
output = graph.stream(new_input, config2)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

---

## 정리: Interrupt

### 핵심 코드

```python
# 1. checkpointer 필수
graph = builder.compile(checkpointer=MemorySaver())

# 2. interrupt_before로 중단 지점 지정
output = graph.stream(input, config, interrupt_before=["model"])

# 3. 상태 확인
state = graph.get_state(config)
print(state.values)  # 현재 상태
print(state.next)    # 다음 실행할 노드

# 4. 재개 (저장된 상태에서)
output = graph.stream(None, config)

# 5. 새 입력으로 재개
output = graph.stream(new_input, config)
```

### interrupt_before vs interrupt_after

| 옵션 | 중단 시점 | 사용 시나리오 |
|------|----------|---------------|
| **interrupt_before** | 노드 실행 전 | 실행 전 승인 |
| **interrupt_after** | 노드 실행 후 | 결과 검토 후 진행 |

### 사용 사례

| 사례 | 설명 |
|------|------|
| 도구 승인 | 위험한 도구 실행 전 확인 |
| 결제 확인 | 결제 API 호출 전 승인 |
| 데이터 삭제 | 삭제 작업 전 확인 |
| Human-in-the-loop | 사람의 검토 필요 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## 다음 단계

**상태 수정** 및 **히스토리에서 Fork**하는 방법을 배웁니다. (07-11번 노트북)

---
## 코드 8-7~8-11 고급 상태 관리: 수정, 재개, Fork

# 고급 상태 관리: 수정, 재개, Fork

이 노트북에서는 **상태 수정, 다양한 재개 방법, 히스토리에서 Fork**하는 고급 기능을 배웁니다.

## 고급 상태 관리 기능

```
┌────────────────────────────────────────────────────────────────────┐
│                    고급 상태 관리                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ get_state: 현재 상태 조회                                    │
│   2️⃣ update_state: 상태 수정                                      │
│   3️⃣ get_state_history: 과거 상태 히스토리 조회                   │
│   4️⃣ Fork: 과거 시점에서 분기하여 새로 실행                       │
│                                                                    │
│   활용:                                                            │
│   • 중단 시점에서 상태 수정 후 재개                                │
│   • 잘못된 결정 시 과거로 돌아가기                                 │
│   • What-if 시나리오 테스트                                        │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={'name': tool.name}) for tool in tools],
    embeddings,
).as_retriever()


class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]


def model_node(state: State) -> State:
    selected_tools = [tool for tool in tools if tool.name in state['selected_tools']]
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    return {'messages': res}


def select_tools(state: State) -> State:
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}


builder = StateGraph(State)
builder.add_node('select_tools', select_tools)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'select_tools')
builder.add_edge('select_tools', 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. 상태 수정 후 재개 (update_state)

In [ ]:
input_data = {
    'messages': [
        HumanMessage(
            '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
        )
    ]
}

config = {'configurable': {'thread_id': 'edit-state-1'}}

print("=== 1단계: 중단 ===")
output = graph.stream(input_data, config, interrupt_before=["model"])
for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
print("🛑 model 전에 중단됨\n")

In [ ]:
# 현재 상태 확인
state = graph.get_state(config)

print("=== 2단계: 현재 상태 확인 ===")
print(f"현재 질문: {state.values['messages'][0].content}")
print(f"선택된 도구: {state.values.get('selected_tools', [])}")

In [ ]:
# 상태 수정: 질문 변경
print("=== 3단계: 상태 수정 ===")

# 메시지 복사 후 첫 번째 메시지 변경
updated_messages = state.values['messages'].copy()
updated_messages[0] = HumanMessage(content="영화 보이후드의 촬영 기간은 얼마나 되나요?")

# 상태 업데이트 적용
update = {'messages': updated_messages}
graph.update_state(config, update)

# 수정된 상태 확인
new_state = graph.get_state(config)
print(f"변경된 질문: {new_state.values['messages'][0].content}")

In [ ]:
# 수정된 상태로 재개
print("=== 4단계: 수정된 상태로 재개 ===")

output = graph.stream(None, config)

for chunk in output:
    for node_name, node_output in chunk.items():
        print(f"노드: {node_name}")
        if 'messages' in node_output:
            last_msg = node_output['messages'][-1] if isinstance(node_output['messages'], list) else node_output['messages']
            if hasattr(last_msg, 'content') and last_msg.content:
                print(f"  응답: {str(last_msg.content)[:200]}...")
    print()

print("✅ 완료 (변경된 질문으로 실행됨)")

# 4. 히스토리 조회 (get_state_history)

In [ ]:
# 새 스레드로 전체 실행
config2 = {'configurable': {'thread_id': 'history-1'}}

print("=== 전체 실행 ===")
output = graph.stream(input_data, config2)
for chunk in output:
    for node_name in chunk.keys():
        print(f"노드 실행: {node_name}")

print("\n✅ 실행 완료")

In [ ]:
# 히스토리 조회
print("=== 상태 히스토리 ===")

history = list(graph.get_state_history(config2))

print(f"총 {len(history)}개의 상태 기록\n")

for i, state in enumerate(history):
    print(f"[{i}] 체크포인트: {state.config['configurable'].get('checkpoint_id', 'N/A')[:20]}...")
    print(f"    다음 노드: {state.next}")
    print(f"    메시지 수: {len(state.values.get('messages', []))}")
    print()

# 5. 히스토리에서 Fork

In [ ]:
# 과거 시점에서 다시 실행 (Fork)
print("=== 히스토리에서 Fork ===")

# 중간 시점의 상태 선택 (예: select_tools 완료 후)
if len(history) > 2:
    fork_state = history[2]  # 3번째 상태
    
    print(f"Fork 시점: {fork_state.next}")
    print(f"해당 시점의 config: {fork_state.config}")
    print()
    
    # 해당 시점에서 재실행
    print("=== Fork 지점에서 재실행 ===")
    output = graph.stream(None, fork_state.config)
    
    for chunk in output:
        for node_name in chunk.keys():
            print(f"노드 실행: {node_name}")
    
    print("\n✅ Fork 완료")
else:
    print("히스토리가 충분하지 않습니다.")

---

## 정리: 고급 상태 관리

### 핵심 API

```python
# 1. 현재 상태 조회
state = graph.get_state(config)
print(state.values)  # 상태 값
print(state.next)    # 다음 실행할 노드

# 2. 상태 수정
update = {'messages': new_messages}
graph.update_state(config, update)

# 3. 히스토리 조회
history = list(graph.get_state_history(config))
for state in history:
    print(state.config, state.next)

# 4. Fork (과거 시점에서 재실행)
output = graph.stream(None, history[2].config)
```

### 기능 비교

| 기능 | 용도 | API |
|------|------|-----|
| **get_state** | 현재 상태 확인 | `graph.get_state(config)` |
| **update_state** | 상태 수정 | `graph.update_state(config, update)` |
| **get_state_history** | 히스토리 조회 | `graph.get_state_history(config)` |
| **Fork** | 과거에서 분기 | `graph.stream(None, past_config)` |

### 사용 시나리오

| 시나리오 | 방법 |
|----------|------|
| 질문 수정 후 재실행 | update_state → stream(None) |
| 잘못된 결정 롤백 | get_state_history → Fork |
| What-if 테스트 | Fork로 여러 시나리오 실행 |
| 디버깅 | get_state_history로 각 단계 검토 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## ch08 요약: 고급 기능

| 기능 | 용도 | 핵심 API |
|------|------|----------|
| **Structured Output** | 구조화된 응답 | `with_structured_output()` |
| **Streaming** | 실시간 출력 | `stream()`, `astream_events()` |
| **Interrupt** | 중단/재개 | `interrupt_before`, `stream(None)` |
| **State Management** | 상태 수정/Fork | `get_state()`, `update_state()` |

## 다음 단계

**ch10에서는** RAG과 Agent의 **평가(Evaluation)** 방법을 배웁니다.